# Day 11: Seating System
---
Soalnya capek karena di bagian kedua harus refaktor kembali fungsi yang dipakai di bagian sebelumnya.

In [1]:
inputs = None
with open("input.txt") as file:
    inputs = file.read().strip()

inputs[:30]

'LLL.LLL.LLLLL.LLLLLLLLLLLLLLLL'

---
## Classes

Di sini aku buat 2 *class*, yang pertama *class* ```Grid``` untuk membungkus struktur data *grid* atau *array* 2 dimensi, dan kedua baru *class* untuk menjalankan siklus perubahan kondisinya.

*Class* ```Grid``` nantinya dapat digunakan lagi untuk soal yang menggunakan struktur data 2 dimensi dengan cara diekspor ke file ```.py```

In [2]:
class Grid:
    adjacents = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    def __init__(self, grid = [[]]):
        self.grid = grid
        self.max_row = len(grid)
        self.max_col = len(grid[0])
    
    def get(self, row, col):
        if 0 <= row < self.max_row and 0 <= col < self.max_col:
            return self.grid[row][col]
        return None
    
    def set(self, row, col, value):
        if 0 <= row < self.max_row and 0 <= col < self.max_col:
            self.grid[row][col] = value
            return True
        return False
    
    def get_adjacents_cells(self, row, col):
        return self.get_8_directions(row, col)
    
    def get_8_directions(self, row, col, ignore_func=lambda x: False):
        result = []
        for dy, dx in self.adjacents:
            sx = sy = 0
            while True:
                sx += dx
                sy += dy
                subresult = self.get(row+sy, col+sx)
                if subresult == None or not ignore_func(subresult):
                    result.append(subresult)
                    break
        return result

    def enumerate(self):
        return [
            (row, col, self.get(row,col))
            for row in range(self.max_row) for col in range(self.max_col)
        ]

    def __str__(self):
        return "\n".join("".join(*zip(*row)) for row in self.grid)

    def __repr__(self):
        return str(self)

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            str(self) == str(other)
        )


In [3]:
import copy

class SeatCycle:
    def __init__(self, inputs):
        self.initial_grid = Grid([[ch for ch in row] for row in inputs.strip().split("\n")])

    def next_cycle1(self, previous = None):
        previous = self.initial_grid if previous == None else previous
        result = copy.deepcopy(previous)
        for row, col, value in previous.enumerate():
            if value == "L":
                if previous.get_adjacents_cells(row, col).count("#") == 0:
                    result.set(row, col, "#")
            elif value == "#":
                if previous.get_adjacents_cells(row, col).count("#") >= 4:
                    result.set(row, col, "L")
        return result
    
    def next_cycle2(self, previous = None):
        previous = self.initial_grid if previous == None else previous
        result = copy.deepcopy(previous)
        for row, col, value in previous.enumerate():
            if value == "L":
                if previous.get_8_directions(row, col, lambda x: x == ".").count("#") == 0:
                    result.set(row, col, "#")
            elif value == "#":
                if previous.get_8_directions(row, col, lambda x: x == ".").count("#") >= 5:
                    result.set(row, col, "L")
        return result
    
    def get_stable_state1(self):
        prev_state = None
        while True:
            next_state = self.next_cycle1(prev_state)
            if (next_state == prev_state):
                break
            prev_state = next_state
        return prev_state
    
    def get_stable_state2(self):
        prev_state = None
        while True:
            next_state = self.next_cycle2(prev_state)
            if (next_state == prev_state):
                break
            prev_state = next_state
        return prev_state

---
## Unit Testing

Karena contoh soalnya panjang banget, unit testingnya juga jadi panjang.

In [4]:
test_sc = SeatCycle(
"""
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
)

In [5]:
# After one round of these rules, every seat in the example layout becomes occupied:

grid_1 = test_sc.next_cycle1()
grid_1

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

In [6]:
# After a second round, the seats with four or more occupied adjacent seats become empty again:

grid_2 = test_sc.next_cycle1(grid_1)
grid_2

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##

In [7]:
# At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state!
# Once people stop moving around, you count 37 occupied seats.

str(test_sc.get_stable_state1()).count("#") == 37

True

In [8]:
# For example, the empty seat below would see eight occupied seats:

test_sc2 = SeatCycle(
"""
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
"""
)

str(test_sc2.initial_grid.get_8_directions(4, 3, lambda x: x == ".")).count("#") == 8

True

In [9]:
# The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

test_sc3 = SeatCycle(
"""
.............
.L.L.#.#.#.#.
.............
"""
)

sight = test_sc3.initial_grid.get_8_directions(1, 1, lambda x: x == ".")
str(sight).count("L") == 1 and str(sight).count("#") == 0

True

In [10]:
# The empty seat below would see no occupied seats:

test_sc4 = SeatCycle(
"""
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
"""
)

str(test_sc4.initial_grid.get_8_directions(3, 3, lambda x: x == ".")).count("#") == 0

True

In [11]:

test_sc5 = SeatCycle(
"""
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
)

str(test_sc5.get_stable_state2()).count("#") == 26

True

---
## Part 1
Bagian pertama cukup gunakan ```while``` untuk mencari *state* yang sama setelah ```next_cycle``` dijalankan.

In [12]:
sc = SeatCycle(inputs)

%time str(sc.get_stable_state1()).count("#")

Wall time: 6.77 s


2265

---
## Part 2
Bagian kedua hanya merubah kriterianya dengan mengganti ```ignore_func``` yang disediakan. Aku jadikan 2 *method* berbeda saja daripada harus *passing parameter* ke fungsinya.

In [13]:
%time str(sc.get_stable_state2()).count("#")

Wall time: 8.21 s


2045